In [15]:
#Ignorer les warnings

import warnings
warnings.filterwarnings("ignore")

In [16]:
#Importation des librairies nécessaires

import pandas as pd
import numpy as np
from datetime import datetime
import csv

# Importation des données

In [17]:
#Importation du fichier | MODIFIER LE NOM DU FICHIER A CHAQUE ITERATION (lien en rouge)
data = pd.read_excel(
    r"C:\Users\alyssa.derensy\OneDrive - Groupe Limagrain Holding\Documents\Automatisation_ano\ventes.xlsx")

In [18]:
#Lecture du fichiers ventes pour faire l'automatisation de l'index
ventes = pd.read_csv('table_vente.csv', sep = ";")

# MODE DE L'ALGO : A MODIFIER A CHAQUE UTILISATION

In [19]:
#Enseigne = input("Nom de l'enseigne (sans MAJ ni é):")

#Vérification de l'orthographque
Enseigne = ''

while Enseigne not in ['1', '2', '3','4','5'] :
    Enseigne = input("Nom de l'enseigne (sans MAJ ni é):")

#Mettre la date année/mois des sorties caisses
date = input("Date sous format AAAA/MM :")

Nom de l'enseigne (sans MAJ ni é):3
Date sous format AAAA/MM :2023/08


# Exploration des données

In [20]:
#Visualisation des données
data.head()

,Code Mag,Code Nomenclature,Id,Référence,Nom,Info 1,Info 2,Info 3,Qté.,ean
0,ALB,3.005000e+10,419392.0,3630549.0,Mache baron bio vl 2,BIO,série 2,Sachet,2.0,3.182670e+12
1,ALB,3.005000e+10,419464.0,3193542.0,Carotte nantaise bio vl 2 amel,BIO,série 2,Sachet,1.0,3.182670e+12
2,ALB,3.005000e+10,419470.0,3452642.0,Concombre marketmore bio vl 2,BIO,série 2,Sachet,1.0,3.182670e+12
3,ALB,3.005000e+10,419473.0,3484542.0,Courgette maraichere bio vl 2,BIO,série 2,Sachet,1.0,3.182670e+12
4,ALB,3.005000e+10,419476.0,3567342.0,Epinard geant hiver bio vl 2,BIO,série 2,Sachet,7.0,3.182670e+12


In [21]:
#Détection des valeurs manquantes
data.isnull().sum()

Code Mag                2
Code Nomenclature       3
Id                      3
Référence               3
Nom                     3
Info 1               3696
Info 2                 78
Info 3                140
Qté.                    3
ean                     3
dtype: int64

In [22]:
ventes.tail()

,id,Gencode,Code_article,Volume,Valeur HT,Valeur TTC,PDV,date,Enseigne


In [23]:
if ventes.duplicated().sum() > 0 :
    raise SystemExit("DOUBLON DANS LES VENTES")

In [24]:
#Vérification du principe d'unicité du fichier des ventes
if ventes['id'].duplicated().sum() > 0 :
    raise SystemExit("Principe d'unicité non respecté : id ")

# Mise en forme et traitement des données 

In [25]:
tab = pd.DataFrame()

In [26]:
#Pré-traitement, suppression des lignes vides et des lignes inutiles

if Enseigne == "1" :
    
    #Supprime les lignes sans volumes ni stock | Gain de mémoire
    data.drop(data[(data['Qté Vendue Période N'] == 0)].index,inplace = True)
    
    #Supprime la ligne Totaux
    data = data.dropna(subset=['EAN13'])
    
elif Enseigne == "2" :
    
     #Supprime les lignes sans volumes ni stock | Gain de mémoire
    data.drop(data[(data[' QTE VENTE '] == 0)].index,inplace = True)
    
    #Supression de la ligne totaux à la fin
    data.drop(data[data['CODE ARTICLE '] == ' TOTAUX '].index,inplace = True)
    
    #Suppression de la dernière ligne vide mais avec des espaces à l'intérieur
    data.drop(data[data[' CODE ARTICLE FRN '] == '  '].index,inplace = True)
    data = data.dropna()
    
elif Enseigne == '3' :
    data = data.dropna(subset = ['Référence'])
    
elif Enseigne == '4' :
    data = data.dropna(subset = ['EAN Article'])
    
    #Si des milliers présent : espace insécable
    data['CA HT'] = data['CA HT'].astype(str)
    data['CA HT'] = data['CA HT'].str.replace('\u00A0', '')
    data['CA HT'] = data['CA HT'].str.replace(' ', '')
    data['CA HT'] = data['CA HT'].str.replace('€', '')
    data['CA HT'] = data['CA HT'].str.replace(',', '.')
    
    data['CA HT'] = data['CA HT'].astype(float)
    
elif Enseigne == '5' :
    data.columns = data.iloc[0]
    data.dropna(subset = ['CD_ARTICLE'], inplace = True)
    data.drop(data[(data.index == 0)].index,inplace = True)
    
#Stop le programme si erreur
else :
    raise SystemExit("Stop right there!")

In [27]:
#Création des nouvelles données pour la base de données en fonction de l'enseigne des données

#Itération pour l'enseigne bricomarché
if Enseigne == "1" :
    
                                 
    tab['Gencode'] = data['EAN13']
    tab['Code_article'] = np.nan
    tab['Volume'] = data['Qté Vendue Période N']
    tab['Valeur HT'] = np.nan
    tab['Valeur TTC'] = data['CATTC Vendue Période N']
    tab['PDV'] = data['PDV']
    tab['date'] = date
    tab['Enseigne'] = "1"
    
    m = tab['Volume'] > 0
    
    new = tab.groupby('Gencode').agg({
                                      'Code_article':'first',
                                      'Volume':'sum',
                                      'Valeur HT': 'sum',
                                      'Valeur TTC' : 'sum',
                                      'PDV' : lambda x: x.where(m).count() ,
                                      'date' : 'first',
                                      'Enseigne' : 'first'})
    
    new.reset_index(inplace= True)
    new.insert(0,'id', np.nan) 
    
    #Formate la colonne date en DATE et non en STRING
    new['date'] = pd.to_datetime(new['date'],format='%Y/%m')

    
#Itération pour l'enseigne 2 
elif Enseigne == "2" :
    
    #Supprimer les deux dernières lignes : Le total + une ligne vide
    #data.drop(data.tail(2).index,inplace = True)
    
    tab['Gencode'] = data[' CODE EAN ']
    tab['Code_article'] = data['CODE ARTICLE ']
    tab['Volume'] = data[' QTE VENTE '].astype(float)
    tab['Valeur HT'] = data[' CA HT '].astype(float)
    tab['Valeur TTC'] = np.nan
    tab['PDV'] = np.nan
    tab['date'] = date
    tab['Enseigne'] = "2"
    
    new = tab.groupby('Gencode').agg({
                                      'Code_article':'first',
                                      'Volume':'sum',
                                      'Valeur HT': 'sum',
                                      'Valeur TTC' : 'sum',
                                      'PDV' : 'count' ,
                                      'date' : 'first',
                                      'Enseigne' : 'first'})
    
    new.reset_index(inplace= True)
    new.insert(0,'id', np.nan) 
    
   #Formate la colonne date en DATE et non en STRING
    tab['date'] = pd.to_datetime(tab['date'],format='%Y/%m')

    
elif Enseigne == '3' :
    tab['Gencode'] = data['ean'].astype(str)
    tab['Code_article'] = data['Référence'].astype(str)
    tab['Volume'] = data['Qté.']
    tab['Valeur HT'] = np.nan
    tab['Valeur TTC'] = np.nan
    tab['PDV'] = data['Code Mag']
    tab['date'] = date
    tab['Enseigne'] = "3"
        
    m = tab['Volume'] > 0
    
    new = tab.groupby('Gencode').agg({'Code_article':'first',
                                      'Volume':'sum',
                                      'Valeur HT': 'sum',
                                      'Valeur TTC' : 'sum',
                                      'PDV' : lambda x: x.where(m).count() ,
                                      'date' : 'first',
                                      'Enseigne' : 'first'})
    
    new.reset_index(inplace= True)
    new.insert(0,'id', np.nan) 
    
   #Formate la colonne date en DATE et non en STRING
    new['date'] = pd.to_datetime(tab['date'],format='%Y/%m')
    new['Gencode'] =  new['Gencode'].str.replace('.0','')
    new['Code_article'] =  new['Code_article'].str.replace('.0','')
    
elif Enseigne == '4' :
    
    tab['Gencode'] = data['EAN Article'].astype(str)
    tab['Code_article'] = np.nan
    tab['Volume'] = data['Quantité Vente']
    tab['Valeur HT'] = data['CA HT'].astype(float)
    tab['Valeur TTC'] = np.nan
    tab['PDV'] = data['GLN Magasin']
    tab['date'] = date
    tab['Enseigne'] = "4"
    
    m = tab['Volume'] > 0
    
    new = tab.groupby('Gencode').agg({
                                      'Code_article':'first',
                                      'Volume':'sum',
                                      'Valeur HT': 'sum',
                                      'Valeur TTC' : 'sum',
                                      'PDV' : lambda x: x.where(m).count() ,
                                      'date' : 'first',
                                      'Enseigne' : 'first'})
    
    new.reset_index(inplace= True)
    new.insert(0,'id', np.nan)
    new['Gencode'] =  new['Gencode'].str.replace('.0','')
    
    #Formate la colonne date en DATE et non en STRING
    new['date'] = pd.to_datetime(new['date'],format='%Y/%m')
    
elif Enseigne == '5' :
    tab['Gencode'] = data['CD_ARTICLE'].astype(str)
    tab['Code_article'] = np.nan
    tab['Volume'] = data['QTE'].astype(float)
    tab['Valeur HT'] = np.nan
    tab['Valeur TTC'] = data['CA_TTC'].astype(float)
    tab['PDV'] = np.nan
    tab['date'] = date
    tab['Enseigne'] = "5"
    
    m = tab['Volume'] > 0
    
    new = tab.groupby('Gencode').agg({
                                      'Code_article':'first',
                                      'Volume':'sum',
                                      'Valeur HT': 'sum',
                                      'Valeur TTC' : 'sum',
                                      'PDV' : lambda x: x.where(m).count() ,
                                      'date' : 'first',
                                      'Enseigne' : 'first'})
    
    new.reset_index(inplace= True)
    new.insert(0,'id', np.nan)
    new['Gencode'] =  new['Gencode'].str.replace('.0','')
    
    #Formate la colonne date en DATE et non en STRING
    new['date'] = pd.to_datetime(new['date'],format='%Y/%m')
    

    
#Stop le programme si aucune enseigne répertorié est décrite
else :
    raise SystemExit("Stop right there!")

# Vérification de l'intégrité des données

In [28]:
#Stop le système si erreurs et donc perte de données pour les volummes

a = round(new['Volume'].sum())

if Enseigne == "1" :
    
    b = data['Qté Vendue Période N'].sum().round()
        
elif Enseigne == '2' :

    b = round(data[' QTE VENTE '].sum())
        
elif Enseigne == '3' :

    b = round(data['Qté.'].sum())
      
elif Enseigne == '4' :

    b = round(data['Quantité Vente'].sum())
    
elif Enseigne == '5' :
    b = round(data['QTE'].sum())
            
#Stop le programme si aucune enseigne répertorié est décrite
else :
    raise SystemExit("Stop right there!")
    
#Vérification de l'intégrité des données des volumes
if a == b : print('Ok !')
        
else :
    print("Volume :",a,"Volume data brutes :",b)
    raise SystemExit("Volume différent !") 

Ok !


In [29]:
#Stop le système si erreurs et donc perte de données pour le CA

c = new['Valeur TTC'].sum().round()

if Enseigne == "1" :
    
    d = data['CATTC Vendue Période N'].sum().round()
    
            
elif Enseigne == "2" :
    
    c = new['Valeur HT'].sum().round()
    d = round(data[' CA HT '].sum())
    

elif Enseigne == '3' :
    
    d = 0
    print('Pas de valeur, Ok !')
    
    
elif Enseigne == "4" :
    
    c = round(new['Valeur HT'].sum())
    d = round(data['CA HT'].sum())
    
elif Enseigne == '5' :
    d = round(data['CA_TTC'].sum())
    
            
#Stop le programme si aucune enseigne répertorié est décrite
else :
    raise SystemExit("Stop right there!")

#Vérification de l'intégration des données pour le CA    
if c == d :
    print('Ok !')

else:
    print("Valeur :",c,"Valeur data brutes :",d)
        
    raise SystemExit("Valeurs différentes !")

Pas de valeur, Ok !
Ok !


# Gestion de l'index

In [30]:
new['id'] = pd.RangeIndex(start = len(ventes), stop = len(new)+len(ventes),step = 1)

In [31]:
new.tail()

,id,Gencode,Code_article,Volume,Valeur HT,Valeur TTC,PDV,date,Enseigne
469,469,3508760548185,4469741,102.0,0.0,0.0,43,2023-08-01,3
470,470,3508760548192,4464841,77.0,0.0,0.0,32,2023-08-01,3
471,471,3508760548208,4464843,155.0,0.0,0.0,54,2023-08-01,3
472,472,3508760548215,4464845,27.0,0.0,0.0,18,2023-08-01,3
473,473,3508760548222,4477423,62.0,0.0,0.0,32,2023-08-01,3


# Exportation des données en fichier CSV

In [32]:
# Exportation Rajoute sur un fichier existant (mode : append), sans le nom des colonnes (header)
#rajout des quote pour les str (gencode) pour éviter les effacements de formats
header = False
new.to_csv('table_vente.csv', index = False, mode = "a",header = header, sep = ';',decimal = ",",quoting=csv.QUOTE_NONNUMERIC)